# 官方BaseLine

In [1]:
import os, sys, glob, shutil, json

from torchvision.models import ResNet18_Weights

os.environ["CUDA_VISIBLE_DEVICES"] = '0'
import cv2

from PIL import Image
import numpy as np

from tqdm import tqdm, tqdm_notebook

# %pylab inline

# 使用matplotlib进行绘图
import matplotlib.pyplot as plt
%matplotlib inline

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

In [5]:
from code.SVHNDataset import SVHNDataset

In [6]:
train_path = glob.glob('./tcdata/mchar_train/*.png')
train_path.sort()
train_json = json.load(open('./tcdata/mchar_train.json'))
train_label = [train_json[x]['label'] for x in train_json]
print(len(train_path), len(train_label))

train_loader = torch.utils.data.DataLoader(
    SVHNDataset(train_path, train_label,
                transforms.Compose([
                    transforms.Resize((64, 128)),
                    transforms.RandomCrop((60, 120)),
                    transforms.ColorJitter(0.3, 0.3, 0.2),
                    transforms.RandomRotation(10),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])),
    batch_size=128,
    shuffle=True,
    num_workers=10,
)

val_path = glob.glob('./tcdata/mchar_val/*.png')
val_path.sort()
val_json = json.load(open('./tcdata/mchar_val.json'))
val_label = [val_json[x]['label'] for x in val_json]
print(len(val_path), len(val_label))

val_loader = torch.utils.data.DataLoader(
    SVHNDataset(val_path, val_label,
                transforms.Compose([
                    transforms.Resize((60, 120)),
                    # transforms.ColorJitter(0.3, 0.3, 0.2),
                    # transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])),
    batch_size=128,
    shuffle=False,
    num_workers=10,
)

30000 30000
10000 10000


ResNet18模型进行特征提取

In [7]:
class SVHN_Model1(nn.Module):
    def __init__(self):
        super(SVHN_Model1, self).__init__()

        model_conv = models.resnet18(weights = ResNet18_Weights.IMAGENET1K_V1)
        model_conv.avgpool = nn.AdaptiveAvgPool2d(1)
        # 移除预训练ResNet-18模型的最后一个全连接层，并保留其余层作为特征提取器
        model_conv = nn.Sequential(*list(model_conv.children())[:-1])
        self.cnn = model_conv

        self.fc1 = nn.Linear(512, 11)
        self.fc2 = nn.Linear(512, 11)
        self.fc3 = nn.Linear(512, 11)
        self.fc4 = nn.Linear(512, 11)
        self.fc5 = nn.Linear(512, 11)

    def forward(self, img):
        feat = self.cnn(img)
        # print(feat.shape)
        feat = feat.view(feat.shape[0], -1)
        c1 = self.fc1(feat)
        c2 = self.fc2(feat)
        c3 = self.fc3(feat)
        c4 = self.fc4(feat)
        c5 = self.fc5(feat)
        return c1, c2, c3, c4, c5

s

In [10]:
def train(train_loader, model, criterion, optimizer, epoch,use_cuda=True):
    # 切换模型为训练模式
    model.train()
    train_loss = []# 用于记录每个 batch 的损失值

    for i, (input, target) in enumerate(train_loader):
        # 将数据转移到 GPU（如果可用）
        if use_cuda:
            input = input.cuda()
            target = target.cuda().long()

        # 前向传播：模型输出 5 个数字的预测结果
        c0, c1, c2, c3, c4 = model(input)
         # 计算总损失：每个数字预测结果与对应标签的损失之和
        loss = criterion(c0, target[:, 0]) + \
                criterion(c1, target[:, 1]) + \
                criterion(c2, target[:, 2]) + \
                criterion(c3, target[:, 3]) + \
                criterion(c4, target[:, 4])

        # loss /= 6
        # 反向传播与优化
        optimizer.zero_grad() # 清空梯度缓存
        loss.backward()  # 计算梯度
        optimizer.step()  # 更新模型参数

        train_loss.append(loss.item())# 记录当前 batch 的损失值
    # 返回当前 epoch 的平均训练损失
    return np.mean(train_loss)

def validate(val_loader, model, criterion,use_cuda=True):
    # 切换模型为预测模型 （禁用 BatchNorm 和 Dropout）
    model.eval()
    val_loss = []

    # 不记录模型梯度信息
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            if use_cuda:
                input = input.cuda()
                target = target.cuda().long()

            c0, c1, c2, c3, c4 = model(input)
            loss = criterion(c0, target[:, 0]) + \
                    criterion(c1, target[:, 1]) + \
                    criterion(c2, target[:, 2]) + \
                    criterion(c3, target[:, 3]) + \
                    criterion(c4, target[:, 4])
            # loss /= 6
            val_loss.append(loss.item())
    # 返回验证集的平均损失
    return np.mean(val_loss)

def predict(test_loader, model, tta=10,use_cuda=True):
    model.eval()
    test_pred_tta = None

    # TTA 次数 # 测试时增强（Test-Time Augmentation）循环
    for _ in range(tta):
        test_pred = []

        with torch.no_grad():# 禁用梯度计算
            for i, (input, target) in enumerate(test_loader):
                if use_cuda:
                    input = input.cuda()
                # 前向传播得到 5 个数字的预测结果
                c0, c1, c2, c3, c4 = model(input)
                # 将预测结果从 GPU 转移到 CPU 并转换为 NumPy 数组
                if use_cuda:
                    output = np.concatenate([
                        c0.data.cpu().numpy(),
                        c1.data.cpu().numpy(),
                        c2.data.cpu().numpy(),
                        c3.data.cpu().numpy(),
                        c4.data.cpu().numpy()], axis=1)
                else:
                    output = np.concatenate([
                        c0.data.numpy(),
                        c1.data.numpy(),
                        c2.data.numpy(),
                        c3.data.numpy(),
                        c4.data.numpy()], axis=1)

                test_pred.append(output)

        test_pred = np.vstack(test_pred)
        if test_pred_tta is None:
            test_pred_tta = test_pred
        else:
            test_pred_tta += test_pred

    return test_pred_tta

## 训练和验证


In [11]:
model = SVHN_Model1()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 0.001)
best_loss = 1000.0

# 是否使用GPU
use_cuda = True
if use_cuda:
    model = model.cuda()

for epoch in range(10):
     # 训练阶段
    train_loss = train(train_loader, model, criterion, optimizer, epoch,use_cuda=use_cuda)
     # 验证阶段
    val_loss = validate(val_loader, model, criterion,use_cuda=use_cuda)

    # 获取真实标签（将数字序列转为字符串，如 [1,2,3,4,5] → "12345"）
    val_label = [''.join(map(str, x)) for x in val_loader.dataset.img_label]
    val_predict_label = predict(val_loader, model, 1,use_cuda=use_cuda)
    val_predict_label = np.vstack([
        val_predict_label[:, :11].argmax(1), # 第1个数字的预测(找出0-11列最大值的索引)
        val_predict_label[:, 11:22].argmax(1), # 第2个数字
        val_predict_label[:, 22:33].argmax(1), # 第3个数字
        val_predict_label[:, 33:44].argmax(1),  # 第4个数字
        val_predict_label[:, 44:55].argmax(1),  # 第5个数字
    ]).T
    val_label_pred = []
    for x in val_predict_label:
        val_label_pred.append(''.join(map(str, x[x!=10])))

    val_char_acc = np.mean(np.array(val_label_pred) == np.array(val_label))

    print('Epoch: {0}, Train loss: {1} \t Val loss: {2}'.format(epoch, train_loss, val_loss))
    print('Val Acc', val_char_acc)
    # 记录下验证集精度
    if val_loss < best_loss:
        best_loss = val_loss
        print('Find better model in Epoch {0}, saving model.'.format(epoch))
        torch.save(model.state_dict(), 'model/model.pt')

Epoch: 0, Train loss: 3.6116945246060688 	 Val loss: 3.4786352128982543
Val Acc 0.3509
Find better model in Epoch 0, saving model.
Epoch: 1, Train loss: 2.2811499365965524 	 Val loss: 3.1218612871170044
Val Acc 0.4122
Find better model in Epoch 1, saving model.
Epoch: 2, Train loss: 1.901327342669169 	 Val loss: 2.7649501342773437
Val Acc 0.4704
Find better model in Epoch 2, saving model.
Epoch: 3, Train loss: 1.696124336242676 	 Val loss: 2.7980010838508607
Val Acc 0.4919
Epoch: 4, Train loss: 1.5408520084222157 	 Val loss: 2.733173876285553
Val Acc 0.5037
Find better model in Epoch 4, saving model.
Epoch: 5, Train loss: 1.4317081383069357 	 Val loss: 2.6738844499588015
Val Acc 0.5063
Find better model in Epoch 5, saving model.
Epoch: 6, Train loss: 1.3460763751665752 	 Val loss: 2.511698058605194
Val Acc 0.547
Find better model in Epoch 6, saving model.
Epoch: 7, Train loss: 1.2576262759367625 	 Val loss: 2.515715271472931
Val Acc 0.5393
Epoch: 8, Train loss: 1.1871500625212987 	 Val

预测并生成提交文件

In [12]:
test_path = glob.glob('./tcdata//mchar_test_a/*.png')
test_path.sort()
# test_json = json.load(open('./tcdata/mchar_test.json'))
test_label = [[1]] * len(test_path)
print(len(test_path), len(test_label))

test_loader = torch.utils.data.DataLoader(
    SVHNDataset(test_path, test_label,
                transforms.Compose([
                    transforms.Resize((70, 140)),
                    # transforms.RandomCrop((60, 120)),
                    # transforms.ColorJitter(0.3, 0.3, 0.2),
                    # transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])),
    batch_size=128,
    shuffle=False,
    num_workers=10,
)



40000 40000


In [15]:
# 加载保存的最优模型
model.load_state_dict(torch.load('./model/model.pt'))

test_predict_label = predict(test_loader, model, 1)
print(test_predict_label.shape)

test_label = [''.join(map(str, x)) for x in test_loader.dataset.img_label]
test_predict_label = np.vstack([
    test_predict_label[:, :11].argmax(1),
    test_predict_label[:, 11:22].argmax(1),
    test_predict_label[:, 22:33].argmax(1),
    test_predict_label[:, 33:44].argmax(1),
    test_predict_label[:, 44:55].argmax(1),
]).T

test_label_pred = []
for x in test_predict_label:
    test_label_pred.append(''.join(map(str, x[x!=10])))

import pandas as pd
df_submit = pd.read_csv('./tcdata/mchar_sample_submit_A.csv')
df_submit['file_code'] = test_label_pred
df_submit.to_csv('./prediction_result/submit_baseline.csv', index=None)

(40000, 55)
